In [1]:
import copy
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt

In [2]:
s = 2
r = 0.04
rho = 0.05
zmean = 1
Corr = 0.9
the = -np.log(Corr)
sig2 = 0.05
Var = sig2/(2*the)

I = 100
amin = -0.1
amax = 30
a = np.linspace(amin,amax,I).reshape((I,1))
da = (amax-amin)/(I-1)

J = 40
zmin = 0.4
zmax = 1.5
z = np.linspace(zmin,zmax,J).reshape((J,1))
dz = (zmax-zmin)/(J-1)
dz2 = dz**2

aa = a * np.ones((I, J))
zz = np.ones((I, J)) * z.T

mu = the*(zmean - z).reshape((1,J))
s2 = sig2*np.ones((1,J))

maxit= 100
crit = 10**(-6)
Delta = 1000

Vaf = np.zeros((I,J))
Vab = np.zeros((I,J))
Vzf = np.zeros((I,J))
Vzb = np.zeros((I,J))
Vzz = np.zeros((I,J))
c = np.zeros((I,J))

chi =  (-np.minimum(mu,0)/dz + s2/(2*dz2)).reshape((J,1))
yy =  (np.minimum(mu,0)/dz - np.maximum(mu,0)/dz - s2/dz2).reshape((J,1))
zeta = (np.maximum(mu,0)/dz + s2/(2*dz2)).reshape((J,1))

In [3]:
# This will be the upperdiagonal of the B_switch
updiag = np.zeros((I,1))
for j in range(J):
    updiag = np.vstack((updiag, np.tile(zeta[j], (I,1))))

# This will be the center diagonal of the B_switch
centdiag = np.tile(chi[0]+yy[0],(I,1))
for j in range(1,J-1):
    centdiag = np.vstack((centdiag, np.tile(yy[j], (I,1))))
centdiag = np.vstack((centdiag, np.tile(yy[J-1]+zeta[J-1], (I,1))))

# This will be the lower diagonal of the B_switch
lowdiag = np.tile(chi[1], (I,1))
for j in range(2,J):
    lowdiag = np.vstack((lowdiag, np.tile(chi[j], (I,1))))

# Add up the upper, center, and lower diagonal into a sparse matrix
Bswitch = scipy.sparse.spdiags(centdiag[:,0], 0, I*J, I*J)\
          + scipy.sparse.spdiags(lowdiag[:,0], -I, I*J, I*J)\
          + scipy.sparse.spdiags(updiag[:,0], I, I*J, I*J)

In [23]:
print(Bswitch.getrow(-1))

  (0, 3899)	33.29337443046311
  (0, 3999)	-33.29337443046311


In [4]:
# Initial guess
v0 = (zz + r*aa)**(1-s)/(1-s)/rho
v = copy.deepcopy(v0)

maxit = 30
for n in range(maxit):
    V = copy.deepcopy(v)
    Vaf[0:I-1,:] = (V[1:I,:]-V[0:I-1,:])/da
    re = (z + r*amax)**(-s)
    Vaf[I-1,:] = re.reshape((J,))
    Vab[1:I,:] = (V[1:I,:]-V[0:I-1,:])/da
    re = (z + r*amin)**(-s) 
    Vab[0,:] = re.reshape((J,))
    I_concave = Vab > Vaf
    
    cf = Vaf**(-1/s)
    sf = zz + r*aa - cf
    cb = Vab**(-1/s)
    sb = zz + r*aa - cb
    c0 = zz + r*aa
    Va0 = c0**(-s)
    If = np.where(sf>0, 1, 0)
    Ib = np.where(sb<0, 1, 0)
    I0 = (1-If-Ib)
    Va_Upwind = Vaf*If + Vab*Ib + Va0*I0
    c = Va_Upwind**(-1/s)
    u = c**(1-s)/(1-s)

    # CONSTRUCT MATRIX A
    X = -np.minimum(sb,0)/da   
    Y = -np.maximum(sf,0)/da + np.minimum(sb,0)/da
    Z = np.maximum(sf,0)/da
    Z = Z.reshape(-1,order='F')
    A_up = scipy.sparse.spdiags(np.concatenate([np.array([0]),Z]),1,I*J,I*J)
    Y = Y.reshape(-1,order='F')
    A_diag = scipy.sparse.spdiags(Y, 0, I*J, I*J)
    X = X.reshape(-1,order='F')
    X = np.roll(X, -1)
    A_down = scipy.sparse.spdiags(X, -1, I*J, I*J)
    AA = A_down + A_diag + A_up
    A = AA + Bswitch
    B = (1/Delta + rho)*scipy.sparse.eye(I*J) - A

    u_stack = u.reshape(-1,order='F')
    V_stack = V.reshape(-1,order='F')
    b = u_stack + V_stack/Delta

    V_stack = scipy.sparse.linalg.spsolve(B, b)
    v = V_stack.reshape((I,J), order='F')

In [5]:
AT = copy.deepcopy(A.T).todense()
b = np.zeros((I*J,1))
i_fix = 0
b[i_fix] = 0.001
row = np.concatenate((np.array([1]), np.zeros((I*J-1))))
AT[i_fix,:] = row
gg = scipy.sparse.linalg.spsolve(scipy.sparse.csr_matrix(AT), b)
g_sum = gg@np.ones((I*J))*da*dz
gg = gg/g_sum
g = gg.reshape((I,J), order='F')

In [6]:
ss = zz + r*aa -c

In [7]:
for i in g:
    print(i)

[0.20920037 0.22133533 0.23097185 0.23804244 0.24251714 0.24440942
 0.24378033 0.24074092 0.23545276 0.22812647 0.21901816 0.20842372
 0.19667094 0.18387296 0.17038588 0.15653781 0.14262258 0.12889591
 0.11557345 0.10283059 0.09080353 0.07959161 0.06909188 0.0594983
 0.05088103 0.04321591 0.03646195 0.03056561 0.02546479 0.02109226
 0.01737863 0.01425477 0.01165379 0.00951257 0.00777288 0.00638218
 0.00529417 0.00446911 0.00387406 0.00348306]
[0.12849602 0.13726464 0.14574172 0.15379745 0.16130245 0.16812771
 0.17414533 0.17923004 0.1832613  0.18612599 0.18772171 0.18796074
 0.1867745  0.18435534 0.18069232 0.17573756 0.16946792 0.16202853
 0.15359556 0.14436637 0.13454982 0.12435698 0.11371568 0.10305276
 0.09265338 0.0826635  0.07320193 0.06436013 0.05620316 0.04877167
 0.04208453 0.03614194 0.03092882 0.02641826 0.02257482 0.01935769
 0.01672356 0.01462918 0.01303352 0.01189975]
[0.11109098 0.11881663 0.12645934 0.1339249  0.14111524 0.14792961
 0.15426595 0.16002255 0.16509996 0.16